# Consumer Cellular ML Models

This notebook trains and deploys ML models for:
1. **Churn Prediction** - Predict which customers are likely to leave
2. **Customer Lifetime Value (LTV)** - Predict customer value over time

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.ml.registry import Registry
from snowflake.ml.modeling.ensemble import RandomForestClassifier, GradientBoostingRegressor
from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.pipeline import Pipeline
import pandas as pd

session = get_active_session()
print("Session connected")

In [ ]:
%%sql -r churn_features
SELECT 
    c.CUSTOMER_ID,
    c.AGE,
    DATEDIFF(MONTH, c.SIGNUP_DATE, CURRENT_DATE()) AS TENURE_MONTHS,
    CASE WHEN c.CUSTOMER_SEGMENT = 'Senior' THEN 1 ELSE 0 END AS IS_SENIOR,
    CASE WHEN c.AARP_MEMBER THEN 1 ELSE 0 END AS IS_AARP,
    p.MONTHLY_PRICE AS PLAN_PRICE,
    CASE WHEN p.UNLIMITED_DATA THEN 1 ELSE 0 END AS HAS_UNLIMITED,
    COALESCE(AVG(u.DATA_USED_GB), 0) AS AVG_DATA_USAGE,
    COALESCE(AVG(u.TALK_MINUTES_USED), 0) AS AVG_TALK_MINS,
    COALESCE(SUM(u.OVERAGE_CHARGES), 0) AS TOTAL_OVERAGE,
    COUNT(DISTINCT t.TICKET_ID) AS SUPPORT_TICKETS,
    COALESCE(AVG(t.SATISFACTION_SCORE), 3) AS AVG_CSAT,
    CASE WHEN s.STATUS = 'Cancelled' THEN 1 ELSE 0 END AS CHURNED
FROM CCI_INTELLIGENCE.RAW.CUSTOMERS c
LEFT JOIN CCI_INTELLIGENCE.RAW.SUBSCRIPTIONS s ON c.CUSTOMER_ID = s.CUSTOMER_ID
LEFT JOIN CCI_INTELLIGENCE.RAW.PLANS p ON s.PLAN_ID = p.PLAN_ID
LEFT JOIN CCI_INTELLIGENCE.RAW.USAGE_DATA u ON c.CUSTOMER_ID = u.CUSTOMER_ID
LEFT JOIN CCI_INTELLIGENCE.RAW.SUPPORT_TICKETS t ON c.CUSTOMER_ID = t.CUSTOMER_ID
GROUP BY c.CUSTOMER_ID, c.AGE, c.SIGNUP_DATE, c.CUSTOMER_SEGMENT, c.AARP_MEMBER, 
         p.MONTHLY_PRICE, p.UNLIMITED_DATA, s.STATUS

In [ ]:
churn_df = session.create_dataframe(churn_features)

feature_cols = ['AGE', 'TENURE_MONTHS', 'IS_SENIOR', 'IS_AARP', 'PLAN_PRICE', 
                'HAS_UNLIMITED', 'AVG_DATA_USAGE', 'AVG_TALK_MINS', 'TOTAL_OVERAGE',
                'SUPPORT_TICKETS', 'AVG_CSAT']
label_col = 'CHURNED'

train_df, test_df = churn_df.random_split([0.8, 0.2], seed=42)
print(f"Training set: {train_df.count()} rows")
print(f"Test set: {test_df.count()} rows")

In [ ]:
churn_model = RandomForestClassifier(
    input_cols=feature_cols,
    label_cols=[label_col],
    output_cols=['CHURN_PREDICTION'],
    n_estimators=100,
    max_depth=10,
    random_state=42
)

churn_model.fit(train_df)
print("Churn model trained successfully")

In [ ]:
predictions = churn_model.predict(test_df)
results = predictions.select('CHURNED', 'CHURN_PREDICTION').to_pandas()

accuracy = (results['CHURNED'] == results['CHURN_PREDICTION']).mean()
print(f"Churn Model Accuracy: {accuracy:.2%}")

In [ ]:
reg = Registry(session=session, database_name='CCI_INTELLIGENCE', schema_name='ANALYTICS')

churn_mv = reg.log_model(
    model=churn_model,
    model_name='CCI_CHURN_PREDICTOR',
    version_name='v1',
    metrics={'accuracy': accuracy},
    comment='Random Forest churn prediction model for Consumer Cellular customers'
)
print(f"Model registered: CCI_CHURN_PREDICTOR v1")

In [ ]:
%%sql -r ltv_features
SELECT 
    c.CUSTOMER_ID,
    c.AGE,
    DATEDIFF(MONTH, c.SIGNUP_DATE, CURRENT_DATE()) AS TENURE_MONTHS,
    CASE WHEN c.CUSTOMER_SEGMENT = 'Senior' THEN 1 ELSE 0 END AS IS_SENIOR,
    p.MONTHLY_PRICE AS PLAN_PRICE,
    COALESCE(SUM(b.TOTAL_AMOUNT), 0) AS TOTAL_REVENUE,
    COALESCE(AVG(u.DATA_USED_GB), 0) AS AVG_DATA_USAGE,
    COUNT(DISTINCT t.TICKET_ID) AS SUPPORT_TICKETS,
    COALESCE(AVG(f.NPS_SCORE), 7) AS AVG_NPS
FROM CCI_INTELLIGENCE.RAW.CUSTOMERS c
LEFT JOIN CCI_INTELLIGENCE.RAW.SUBSCRIPTIONS s ON c.CUSTOMER_ID = s.CUSTOMER_ID
LEFT JOIN CCI_INTELLIGENCE.RAW.PLANS p ON s.PLAN_ID = p.PLAN_ID
LEFT JOIN CCI_INTELLIGENCE.RAW.BILLING b ON c.CUSTOMER_ID = b.CUSTOMER_ID
LEFT JOIN CCI_INTELLIGENCE.RAW.USAGE_DATA u ON c.CUSTOMER_ID = u.CUSTOMER_ID
LEFT JOIN CCI_INTELLIGENCE.RAW.SUPPORT_TICKETS t ON c.CUSTOMER_ID = t.CUSTOMER_ID
LEFT JOIN CCI_INTELLIGENCE.RAW.CUSTOMER_FEEDBACK f ON c.CUSTOMER_ID = f.CUSTOMER_ID
WHERE s.STATUS = 'Active'
GROUP BY c.CUSTOMER_ID, c.AGE, c.SIGNUP_DATE, c.CUSTOMER_SEGMENT, p.MONTHLY_PRICE

In [ ]:
ltv_df = session.create_dataframe(ltv_features)

ltv_feature_cols = ['AGE', 'TENURE_MONTHS', 'IS_SENIOR', 'PLAN_PRICE', 
                    'AVG_DATA_USAGE', 'SUPPORT_TICKETS', 'AVG_NPS']
ltv_label = 'TOTAL_REVENUE'

ltv_train, ltv_test = ltv_df.random_split([0.8, 0.2], seed=42)
print(f"LTV Training set: {ltv_train.count()} rows")

In [ ]:
ltv_model = GradientBoostingRegressor(
    input_cols=ltv_feature_cols,
    label_cols=[ltv_label],
    output_cols=['PREDICTED_LTV'],
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42
)

ltv_model.fit(ltv_train)
print("LTV model trained successfully")

In [ ]:
ltv_predictions = ltv_model.predict(ltv_test)
ltv_results = ltv_predictions.select('TOTAL_REVENUE', 'PREDICTED_LTV').to_pandas()

from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(ltv_results['TOTAL_REVENUE'], ltv_results['PREDICTED_LTV'])
r2 = r2_score(ltv_results['TOTAL_REVENUE'], ltv_results['PREDICTED_LTV'])

print(f"LTV Model MAE: ${mae:.2f}")
print(f"LTV Model R²: {r2:.3f}")

In [ ]:
ltv_mv = reg.log_model(
    model=ltv_model,
    model_name='CCI_LTV_PREDICTOR',
    version_name='v1',
    metrics={'mae': mae, 'r2': r2},
    comment='Gradient Boosting LTV prediction model for Consumer Cellular customers'
)
print(f"Model registered: CCI_LTV_PREDICTOR v1")

In [ ]:
print("\n=== Registered Models ===")
for model in reg.show_models().collect():
    print(f"- {model['name']}: {model['comment']}")

print("\nML models ready for use with CCI Intelligence Agent!")